In [1]:
# model_name-<feature-size>/
# tensorboard/logs
# initial_loss
# training.png
# fine-tuning.png
# feature-training.csv

# Input

In [2]:
import tensorflow as tf

IMG_SIZE = 224

initial_epochs = 100

num_features = 512

# Percentage of tail of the model to be trainable during fine tuning
fine_tune_at = 0.3

fine_tune_epochs = 50

path_dataset = "datasets/dataset-survivor.csv"

slice_datasource = "3d-dataset-bigger-slices-224"

In [3]:
BATCH_SIZE = IMG_SIZE

IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

In [4]:
keras_models = [
    "Xception",
    "VGG19",
    "ResNet50",
    "ResNet101",
    "InceptionV3",
    "InceptionResNetV2",
    "MobileNetV2",
    "DenseNet169",
    "NASNetMobile"
    
#   VGG16,
#    ResNet152,
#    ResNet50V2,
#    ResNet101V2,
#    ResNet152V2,
#    MobileNet,
#    DenseNet121,
#    DenseNet201,
#    NASNetLarge,
#    EfficientNetB0,
#    EfficientNetB1,
#    EfficientNetB2,
#    EfficientNetB3,
#    EfficientNetB4,
#    EfficientNetB5,
#    EfficientNetB6,
#    EfficientNetB7   
]

# Data Loading

In [5]:
import pandas as pd
import glob
import imageio
import numpy as np

class Dataset():
    
    def __init__(self, path_dataset, slice_datasource):
        self.path_dataset = path_dataset
        self.slice_datasource = slice_datasource

        # Read datasource subject and label
        df = pd.read_csv(path_dataset)
        self.subjects = df.iloc[:,1]
        self.labels = df.iloc[:,2]
        
        # Load all data
        self.__load_all_data()
        
    def __load_all_data(self):
        result = {}
        for subject in self.subjects:
            X = []
            paths = []
            for image_path in glob.glob(f'{self.slice_datasource}/{subject}/*.png'):
                img = imageio.imread(image_path)
                X.append(img.astype(np.float32) / 255.)
                paths.append(image_path)
            result[subject] = (np.array(X), paths)
        self.dataset = result
        
    # Return slices and labels given a train and test index set
    def get_slices(self, train_index, test_index):
        
        X_train = []
        y_train = []
        X_test = []
        y_test = []

        for subject, label in zip(self.subjects[train_index], self.labels[train_index]):
            imgs, path = self.dataset[subject]
            for img in imgs:
                X_train.append(img)
                y_train.append(label)

        for subject, label in zip(self.subjects[test_index], self.labels[test_index]):
            imgs, path = self.dataset[subject]
            for img in imgs:
                X_test.append(img)
                y_test.append(label)

        return X_train, y_train, X_test, y_test    

# Plotting

In [6]:
import matplotlib.pyplot as plt

# f"{model_name}-{num_features}/training.png"
def save_training_plot(history, image_path):

    acc = history.history['acc']
    val_acc = history.history['val_acc']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.ylabel('Accuracy')
    plt.ylim([min(plt.ylim()),1])
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 1, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Cross Entropy')
    # plt.ylim([0,1.0])
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    # plt.show()

    plt.savefig(image_path)

In [7]:
# f'{model_name}/fine-tuning.png'
def save_fine_tunining_plot(history, initial_epochs, history_fine, image_path):
    
    acc = history.history['acc']
    val_acc = history.history['val_acc']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    acc += history_fine.history['acc']
    val_acc += history_fine.history['val_acc']

    loss += history_fine.history['loss']
    val_loss += history_fine.history['val_loss']

    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.ylim([0, 1])
    plt.plot([initial_epochs-1,initial_epochs-1],
              plt.ylim(), label='Start Fine Tuning')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 1, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    # plt.ylim([0, 1.0])
    plt.plot([initial_epochs-1,initial_epochs-1],
             plt.ylim(), label='Start Fine Tuning')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    # plt.show()

    plt.savefig(image_path)

# Load Model

In [8]:
from tensorflow.keras.applications import *

# Feature extraction

In [9]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout, Concatenate, concatenate
from tensorflow.keras.models import Model, Sequential

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

import glob
import imageio
import pandas as pd

class TransferLearningModel():
    
    def __init__(self, model_name, num_features):
        
        self.model_name = model_name
        self.num_features = num_features

        # Download and use a pre-trained
        self.base_model = eval(self.model_name)(weights='imagenet', include_top=False, input_shape=IMG_SHAPE)

        self.base_model.trainable = False

        # Add classification head
        self.global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
        self.feature_batch_average = self.global_average_layer(self.base_model.output)

        # Feature layer
        self.feature_layer = Dense(self.num_features, activation='relu')
        self.feature_batch = self.feature_layer(self.feature_batch_average)

        # Prediction layer
        self.prediction_layer = tf.keras.layers.Dense(1)
        self.prediction_batch = self.prediction_layer(self.feature_batch)

        self.model = tf.keras.Sequential([
            self.base_model,
            self.global_average_layer,
            self.feature_layer,
            self.prediction_layer
        ])

        base_learning_rate = 0.0001

        transfer_learning_optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate)

        self.model.compile(transfer_learning_optimizer,
                           loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                           metrics=['accuracy'])

        self.early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
        self.tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=f"{self.model_name}-{self.num_features}/logs")
        
    def fit(self, X_train, y_train, X_test, y_test, initial_epochs):
        
        self.initial_epochs = initial_epochs
        
        self.history = self.model.fit(X_train, y_train,
                                      epochs=initial_epochs,
                                      callbacks=[self.tensorboard_callback, self.early_stopping_callback],
                                      batch_size=IMG_SIZE,
                                      validation_data=(X_test, y_test))
    
    def save_train_history(self, image_path):
        save_training_plot(self.history, image_path)
        
    def tune(self, X_train, y_train, X_test, y_test, fine_tune_epochs):

        self.fine_tune_epochs = fine_tune_epochs

        self.base_model.trainable = True

        # Let's take a look to see how many layers are in the base model
        print("Number of layers in the base model: ", len(self.base_model.layers))

        fine_tune_at_index = int(len(self.base_model.layers) * (1.0 - fine_tune_at))
        print("We leave unmodified", fine_tune_at_index, "layers")

        # Freeze all the layers before the `fine_tune_at` layer
        for layer in self.base_model.layers[:fine_tune_at_index]:
            layer.trainable =  False

        base_learning_rate = 0.0001

        fine_tuning_optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate/10)

        self.model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  optimizer=fine_tuning_optimizer,
                  metrics=['accuracy'])

        total_epochs = self.initial_epochs + self.fine_tune_epochs

        self.early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
        self.tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=f"{self.model_name}-{self.num_features}/logs")

        self.history_fine = self.model.fit(X_train, y_train,
                                 epochs=total_epochs,
                                 callbacks=[self.tensorboard_callback, self.early_stopping_callback],
                                 batch_size=IMG_SIZE,
                                 initial_epoch = self.history.epoch[-1],
                                 validation_data = (X_test, y_test))
        
    def save_fine_history(self, image_path):
        save_fine_tunining_plot(self.history, self.initial_epochs, self.history_fine, image_path)

    def save_features(self, dataset, train_index, csv_name):
            
        # Feature extractor
        feature_extractor = tf.keras.Sequential([
            self.base_model,
            self.global_average_layer,
            self.feature_layer,
        ])
    
        result = None
        
        for subject, label in zip(dataset.subjects, dataset.labels):
            
            print("Loading:", subject)
            (images, paths) = dataset.dataset[subject]
            
            if len(images) > 0:
                features = feature_extractor.predict([images])

                a = features
                a = np.c_[  paths, a ]
                
                is_train = (subject in dataset.subjects[train_index].tolist())
                
                a = np.c_[ np.repeat(label, features.shape[0]), a ]
                a = np.c_[ np.repeat(is_train, features.shape[0]), a ]
                a = np.c_[ np.repeat(subject, features.shape[0]), a ]

                # subject, label, path, features

                if result is None:
                    result = a
                else:
                    result = np.r_[result, a]
                
        result_df = pd.DataFrame(result)
        result_df = result_df.rename(columns={0: "Subject", 1: "Train", 2: "Label", 3: "Image Path"})
        result_df.to_csv(f"{self.model_name}-{self.num_features}/{csv_name}")

Using TensorFlow backend.


In [10]:
from sklearn.model_selection import KFold

# Load data
dataset = Dataset(path_dataset, slice_datasource)

In [11]:
for model_index in range(len(keras_models)):
    
    index = 0

    model_name = keras_models[model_index]

    kf = KFold(n_splits=3, shuffle=True, random_state=42)

    for train_index, test_index in kf.split(dataset.subjects):

        print("Running index", index)

        # Get slices
        X_train, y_train, X_test, y_test = dataset.get_slices(train_index, test_index)

        print(np.array(X_train).shape)
        print(np.array(y_train).shape)
        print(np.array(X_test).shape)
        print(np.array(y_test).shape)

        # Create model
        model = TransferLearningModel(model_name, num_features)

        # Train
        model.fit(np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test), initial_epochs)

        # Plot
        model.save_train_history(f"{model_name}-{num_features}/training-{index}.png")

        # Save Features
        # model.save_features(dataset, train_index, f"features-training-{index}.csv")

        # Fine tuning
        # model.tune(np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test), fine_tune_epochs)

        # Plot
        # model.save_fine_history(f"{model_name}-{num_features}/fine-tune-{index}.png")

        # Save Features
        # model.save_features(dataset, train_index, f"features-fine-tune-{index}.csv")

        index = index + 1

Running index 0
(176, 224, 224, 3)
(176,)
(108, 224, 224, 3)
(108,)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 176 samples, validate on 108 samples
Epoch 1/100
176/176 [==============================] - 12s 70ms/sample - loss: 0.7210 - acc: 0.3807 - val_loss: 0.8723 - val_acc: 0.3889
Epoch 2/100
176/176 [==============================] - 1s 5ms/sample - loss: 0.6406 - acc: 0.7045 - val_loss: 0.7724 - val_acc: 0.4722
Epoch 3/100
176/176 [==============================] - 1s 4ms/sample - loss: 0.5870 - acc: 0.7216 - val_loss: 0.8010 - val_acc: 0.4444
Epoch 4/100
176/176 [==============================] - 1s 4ms/sample - loss: 0.5506 - acc: 0.8011 - val_loss: 0.7953 - val_acc: 0.4259
Epoch 5/100
176/176 [==============================] - 1s 4ms/sample - loss: 0.5212 - acc: 0.8523 - val_loss: 0.8073 - val_acc: 0.4444
Epoch 6/100
176/176 [=============

Epoch 56/100
176/176 [==============================] - 1s 4ms/sample - loss: 0.0923 - acc: 1.0000 - val_loss: 1.1085 - val_acc: 0.4167
Epoch 57/100
176/176 [==============================] - 1s 5ms/sample - loss: 0.0894 - acc: 1.0000 - val_loss: 1.2270 - val_acc: 0.4074
Epoch 58/100
176/176 [==============================] - 1s 4ms/sample - loss: 0.0868 - acc: 1.0000 - val_loss: 1.1259 - val_acc: 0.4167
Epoch 59/100
176/176 [==============================] - 1s 4ms/sample - loss: 0.0842 - acc: 1.0000 - val_loss: 1.2439 - val_acc: 0.4074
Epoch 60/100
176/176 [==============================] - 1s 4ms/sample - loss: 0.0817 - acc: 1.0000 - val_loss: 1.1439 - val_acc: 0.4167
Epoch 61/100
176/176 [==============================] - 1s 4ms/sample - loss: 0.0792 - acc: 1.0000 - val_loss: 1.2648 - val_acc: 0.4074
Epoch 62/100
176/176 [==============================] - 1s 5ms/sample - loss: 0.0769 - acc: 1.0000 - val_loss: 1.1591 - val_acc: 0.4167
Epoch 63/100
176/176 [==========================

Epoch 16/100
194/194 [==============================] - 1s 3ms/sample - loss: 0.3702 - acc: 0.8351 - val_loss: 0.5981 - val_acc: 0.5778
Epoch 17/100
194/194 [==============================] - 1s 4ms/sample - loss: 0.3593 - acc: 0.9639 - val_loss: 0.6106 - val_acc: 0.5444
Epoch 18/100
194/194 [==============================] - 1s 4ms/sample - loss: 0.3488 - acc: 0.8557 - val_loss: 0.5952 - val_acc: 0.5778
Epoch 19/100
194/194 [==============================] - 1s 4ms/sample - loss: 0.3385 - acc: 0.9639 - val_loss: 0.6031 - val_acc: 0.5111
Epoch 20/100
194/194 [==============================] - 1s 4ms/sample - loss: 0.3286 - acc: 0.8763 - val_loss: 0.5932 - val_acc: 0.6111
Epoch 21/100
194/194 [==============================] - 1s 4ms/sample - loss: 0.3194 - acc: 0.9639 - val_loss: 0.5983 - val_acc: 0.5111
Epoch 22/100
194/194 [==============================] - 1s 4ms/sample - loss: 0.3107 - acc: 0.8918 - val_loss: 0.5916 - val_acc: 0.6444
Epoch 23/100
194/194 [==========================

Epoch 77/100
194/194 [==============================] - 1s 3ms/sample - loss: 0.0779 - acc: 1.0000 - val_loss: 0.6487 - val_acc: 0.6556
Epoch 78/100
194/194 [==============================] - 1s 3ms/sample - loss: 0.0759 - acc: 1.0000 - val_loss: 0.7212 - val_acc: 0.6889
Epoch 79/100
194/194 [==============================] - 1s 4ms/sample - loss: 0.0739 - acc: 1.0000 - val_loss: 0.6593 - val_acc: 0.7000
Epoch 80/100
194/194 [==============================] - 1s 4ms/sample - loss: 0.0718 - acc: 1.0000 - val_loss: 0.7229 - val_acc: 0.6889
Epoch 81/100
194/194 [==============================] - 1s 4ms/sample - loss: 0.0698 - acc: 1.0000 - val_loss: 0.6714 - val_acc: 0.7000
Epoch 82/100
194/194 [==============================] - 1s 4ms/sample - loss: 0.0678 - acc: 1.0000 - val_loss: 0.7259 - val_acc: 0.6889
Epoch 83/100
194/194 [==============================] - 1s 4ms/sample - loss: 0.0660 - acc: 1.0000 - val_loss: 0.6806 - val_acc: 0.7000
Epoch 84/100
194/194 [==========================

Epoch 37/100
198/198 [==============================] - 1s 4ms/sample - loss: 0.1889 - acc: 1.0000 - val_loss: 0.7834 - val_acc: 0.4419
Epoch 38/100
198/198 [==============================] - 1s 4ms/sample - loss: 0.1839 - acc: 1.0000 - val_loss: 0.7757 - val_acc: 0.3605
Epoch 39/100
198/198 [==============================] - 1s 3ms/sample - loss: 0.1789 - acc: 1.0000 - val_loss: 0.7896 - val_acc: 0.4419
Epoch 40/100
198/198 [==============================] - 1s 4ms/sample - loss: 0.1742 - acc: 1.0000 - val_loss: 0.7819 - val_acc: 0.3372
Epoch 41/100
198/198 [==============================] - 1s 3ms/sample - loss: 0.1695 - acc: 1.0000 - val_loss: 0.7962 - val_acc: 0.4419
Epoch 42/100
198/198 [==============================] - 1s 3ms/sample - loss: 0.1651 - acc: 1.0000 - val_loss: 0.7884 - val_acc: 0.3372
Epoch 43/100
198/198 [==============================] - 1s 4ms/sample - loss: 0.1607 - acc: 1.0000 - val_loss: 0.8025 - val_acc: 0.4419
Epoch 44/100
198/198 [==========================

...

KeyboardInterrupt: 

Error in callback <function flush_figures at 0x7ffaa82b7ea0> (for post_execute):


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/events.py", line 88, in trigger
    func(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/pylab/backend_inline.py", line 121, in flush_figures
    return show(True)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/pylab/backend_inline.py", line 43, in show
    metadata=_fetch_figure_metadata(figure_manager.canvas.figure)
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/display.py", line 313, in display
    format_dict, md_dict = format(obj, include=include, exclude=exclude)
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/formatters.py", line 180, in format
    data = formatter(obj)
  File "</usr/local/lib/python3.6/dist-packages/decorator.py:decorator-gen-9>", line 2, in __call__
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/formatters.py", line 224, in catch_format_error
    r = method(self, *args, **kwargs)
  File "/usr/l

KeyboardInterrupt: 

In [12]:
from numba import cuda
cuda.select_device(0)
cuda.close()

In [13]:
from keras import backend as K
K.clear_session()